# Testeo del algoritmo de reconocimiento facial

Finalmente llegamos a la parte donde lo vemos funcionar.

Vamos a volver a prender la cámara, detectar caras y para cada cara utilizar el modelo que ya entrenamos para el reconocimiento.

Para mostrar el reconocimiento de forma más amigable, lo primero que debemos hacer es asignarle un nombre a cada identificador. Para ello crearemos un vector de nombres, donde en cada posición correspondiente a cada identificador pondremos el nombre de la persona. Por ejemplo, como la persona asociada al identificador 1 es _Matías_, armaremos nuestro vector como:
```
names = ['', 'Matias']
```
Notar que debemos dejar el primer elemento vacío, ya que los índices en `Python` comienzan en 0, y nosotros pusimos el primer identificador en 1.

In [1]:
import cv2
import numpy as np
import os

# Lista de nombres
names = ['', 'Matias']

# Inicializamos el detector de caras
faceCascade = cv2.CascadeClassifier("default.xml")

# Inicializamos el reconocedor de caras
reconocedor = cv2.face.LBPHFaceRecognizer_create()
reconocedor.read('trainer.yml')

font = cv2.FONT_HERSHEY_SIMPLEX

id = 0
# Iniciamos la captura de video
cam = cv2.VideoCapture(0)
cam.set(3, 640)
cam.set(4, 480)

# Definimos un tamaño minimo para reconocer caras, para filtrar caras demasiado chicas
minW = 0.1 * cam.get(3)
minH = 0.1 * cam.get(4)

while True:
    ret, img = cam.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detectamos caras
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.2,
        minNeighbors=5,
        minSize=(int(minW), int(minH)),
    )
    
    # Para cada cara detectada
    for (x, y, w, h) in faces:
        # Ponemos un rectángulo alrededor
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        # Ejecutamos el reconocimiento facial
        id, dist = reconocedor.predict(gray[y:y + h, x:x + w])
        # Checkeamos si la probabilidad es muy baja, decimos que no podemos reconocer ninguna cara
        if (dist < 80):
            id = names[id]
        else:
            id = names[0]
        # Superponemos el nombre
        cv2.putText(img, str(id), (x + 5, y - 5), font, 1, (255, 255, 255), 2)

    cv2.imshow('camera', img)
    
    # Mostramos la cámara hasta que se presione ESC
    k = cv2.waitKey(10) & 0xff  
    if k == 27:
        break
        
cam.release()
cv2.destroyAllWindows()

## FIN